In [1]:
from sklearn import datasets
data_breast_cancer = datasets.load_breast_cancer(as_frame=True)

In [2]:
data_breast_cancer.data

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


In [103]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data_breast_cancer.data.loc[:, ["mean texture", "mean symmetry"]], data_breast_cancer.target, test_size=0.2)

from sklearn.metrics import accuracy_score
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

In [104]:
log_clf = LogisticRegression(solver="lbfgs",
                             random_state=42)
dctree_clf =  DecisionTreeClassifier()

neigh_clf = KNeighborsClassifier()

voting_clf_hard = VotingClassifier(
    estimators=[('lr', log_clf),
                ('rf', dctree_clf),
                ('knn', neigh_clf)],
    voting='hard')

voting_clf_soft = VotingClassifier(
    estimators=[('lr', log_clf),
                ('rf', dctree_clf),
                ('knn', neigh_clf)],
    voting='soft')
for clf in ( dctree_clf, log_clf,neigh_clf, voting_clf_hard, voting_clf_soft):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    y_pred_train = clf.predict(X_train)
    print(clf.__class__.__name__,
          accuracy_score(y_train, y_pred_train), accuracy_score(y_test, y_pred))


DecisionTreeClassifier 1.0 0.631578947368421
LogisticRegression 0.7208791208791209 0.7280701754385965
KNeighborsClassifier 0.7648351648351648 0.6754385964912281
VotingClassifier 0.832967032967033 0.6754385964912281
VotingClassifier 0.9626373626373627 0.7192982456140351


LogisticRegression 0.7296703296703296 0.7017543859649122
RandomForestClassifier 1.0 0.7280701754385965
SVC 0.7164835164835165 0.7368421052631579
VotingClassifier 0.8483516483516483 0.7280701754385965
KNeighborsClassifier 0.7802197802197802 0.6578947368421053


In [114]:
from sklearn.ensemble import BaggingClassifier



bag_clf = BaggingClassifier(
DecisionTreeClassifier(), n_estimators=30,
max_samples=100, bootstrap=True)
bag_clf.fit(X_train, y_train)
bag_clf.score(X_test, y_test)



0.7368421052631579

In [115]:
bag_clf = BaggingClassifier(
DecisionTreeClassifier(), n_estimators=30,
max_samples=0.5, bootstrap=True)
bag_clf.fit(X_train, y_train)
bag_clf.score(X_test, y_test)

0.7368421052631579

In [116]:
bag_clf = BaggingClassifier(
DecisionTreeClassifier(), n_estimators=30,
max_samples=100, bootstrap=False)
bag_clf.fit(X_train, y_train)
bag_clf.score(X_test, y_test)

0.7368421052631579

In [117]:
bag_clf = BaggingClassifier(
DecisionTreeClassifier(), n_estimators=30,
max_samples=0.5, bootstrap=False)
bag_clf.fit(X_train, y_train)
bag_clf.score(X_test, y_test)

0.7105263157894737

In [130]:
from sklearn.ensemble import RandomForestClassifier
BaggingClassifier(
RandomForestClassifier(n_estimators=500,
                                 max_leaf_nodes=25), n_estimators=30,
max_samples=0.5, bootstrap=True)
rnd_clf.fit(X_train, y_train)
rnd_clf.score(X_test, y_test)

0.7719298245614035

In [129]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier(n_estimators=100)
scores = cross_val_score(clf, data_breast_cancer.data.loc[:, ["mean texture", "mean symmetry"]], data_breast_cancer.target, cv=5)
print(scores.mean())

0.7152926564198107


In [138]:
from sklearn.ensemble import GradientBoostingRegressor
gbrt = GradientBoostingRegressor(max_depth=13
                                 ,n_estimators=6,learning_rate=1.0)
gbrt.fit(X_train, y_train)
gbrt.score(X_test,y_test)

-0.5026578753194397